In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [4]:
file_path = "file:///home/talentum/test-jupyter/evaluation/Lab2_optional/Listofstartups.csv"
df = spark.read.option("header", "true").csv(file_path)


In [5]:
df.show()

+--------------------+--------------------+--------------------+--------------------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|
+--------------------+--------------------+--------------------+--------------------+
|      ABES Ghaziabad|            Suryansh|           New Delhi|              EdTech|
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech|
|AIC Banasthali Vi...|C2M Internet Indi...|Lucknow, Uttar Pr...|         Retail Tech|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare|
|AIC Pinnacle Entr...|  3DGuru Innovations|   Pune, Maharashtra|             EduTech|
|AIC Pinnacle Entr...|     Gupte Education|   Pune, Maharashtra|    Ed Tech, Defence|
|AIC Pinnacle Entr...|Eldew Digital Pvt...|   Pune, Ma

In [6]:
df.printSchema()

root
 |-- Incubation_Center: string (nullable = true)
 |-- Name_of_startup: string (nullable = true)
 |-- Location of company: string (nullable = true)
 |-- Sector: string (nullable = true)



In [29]:
from pyspark.sql.functions import *


In [10]:
sector_count = df.groupBy("Sector").count().orderBy(F.col("count").desc())
print(f"Sector with most startups: {sector_count.take(1)}")
sector_count.show()

Sector with most startups: [Row(Sector='Healthcare', count=34)]
+--------------------+-----+
|              Sector|count|
+--------------------+-----+
|          Healthcare|   34|
|     ICT Electronics|    5|
|            Agritech|    5|
|Education Technology|    5|
|                 IOT|    4|
|      Digital Health|    4|
|                 IoT|    4|
| Digital Health Tech|    3|
|              EdTech|    3|
|             EduTech|    3|
|          Healthtech|    3|
|              Energy|    2|
|            Fit-Tech|    2|
|               AR/VR|    2|
|                  EV|    2|
|             Fintech|    2|
|        Clean Energy|    2|
|Industrial Automa...|    2|
|Information Techn...|    2|
|           Education|    2|
+--------------------+-----+
only showing top 20 rows



In [30]:
df = df.withColumn("city", split(col("Location of company"), ", ").getItem(0))
df = df.withColumn("state", split(col("Location of company"), ", ").getItem(1))
df.drop('Location of company').show()

+--------------------+--------------------+--------------------+------------+-------------+
|   Incubation_Center|     Name_of_startup|              Sector|        city|        state|
+--------------------+--------------------+--------------------+------------+-------------+
|      ABES Ghaziabad|            Suryansh|              EdTech|   New Delhi|         null|
|AIC Banasthali Vi...|Thinkpods Educati...|             Ed Tech|      Satara|  Maharashtra|
|AIC Banasthali Vi...|Inventiway Soluti...|             HR Tech|      Mumbai|  Maharashtra|
|AIC Banasthali Vi...|C2M Internet Indi...|         Retail Tech|     Lucknow|Uttar Pradesh|
|AIC Pinnacle Entr...|            Wastinno|         agriculture|        Pune|  Maharashtra|
|AIC Pinnacle Entr...|Diabetico - Rise ...|          Healthcare|        Pune|  Maharashtra|
|AIC Pinnacle Entr...|  3DGuru Innovations|             EduTech|        Pune|  Maharashtra|
|AIC Pinnacle Entr...|     Gupte Education|    Ed Tech, Defence|        Pune|  M

In [15]:
df = df.withColumn("state", when(col("Location of company").contains("DIAT,Pune"), "Maharashtra").otherwise(col("state")))
df = df.withColumn("city", when(col("Location of company").contains("DIAT,Pune"), "Pune").otherwise(col("city")))

df = df.withColumn("state", when(col("Location of company").contains("Ulhasnagar"), "Maharashtra").otherwise(col("state")))
df = df.withColumn("city", when(col("Location of company").contains("Ulhasnagar"), "Ulhasnagar").otherwise(col("city")))
df.filter(df.city == 'Ulhasnagar').show()

+--------------------+--------------------+-------------------+---------+----------+-----------+
|   Incubation_Center|     Name_of_startup|Location of company|   Sector|      city|      state|
+--------------------+--------------------+-------------------+---------+----------+-----------+
|Society for Innov...|Develop Train Mai...|         Ulhasnagar|CleanTech|Ulhasnagar|Maharashtra|
+--------------------+--------------------+-------------------+---------+----------+-----------+



In [31]:
    df.groupBy(df.state).count().orderBy(col('count').desc()).show()
#     df.groupBy("Sector").count().orderBy(F.col("count").desc())

+--------------+-----+
|         state|count|
+--------------+-----+
|          null|   69|
|     Karnataka|   35|
|   Maharashtra|   32|
|     Tamilnadu|   24|
| Uttar Pradesh|   24|
|     Telangana|   12|
|         Delhi|    9|
|     Rajasthan|    7|
|       Gujarat|    7|
|    Tamil Nadu|    6|
|       Haryana|    5|
|Andhra Pradesh|    4|
|   West Bengal|    2|
|        Punjab|    1|
|            HP|    1|
|         BIHAR|    1|
|     Hyderabad|    1|
|   IIT Roorkee|    1|
+--------------+-----+



In [17]:
df.filter(df.state == "Maharashtra").show()

+--------------------+--------------------+--------------------+--------------------+-------+-----------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|   city|      state|
+--------------------+--------------------+--------------------+--------------------+-------+-----------+
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech| Satara|Maharashtra|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech| Mumbai|Maharashtra|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture|   Pune|Maharashtra|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare|   Pune|Maharashtra|
|AIC Pinnacle Entr...|  3DGuru Innovations|   Pune, Maharashtra|             EduTech|   Pune|Maharashtra|
|AIC Pinnacle Entr...|     Gupte Education|   Pune, Maharashtra|    Ed Tech, Defence|   Pune|Maharashtra|
|AIC Pinnacle Entr...|Eldew Digital Pvt...|   

In [18]:
df.filter(df.Sector == "Healthcare").count()

34

In [19]:
df.filter((df.city == 'Nashik') | (df.city == 'Pune')).show()    

+--------------------+--------------------+-------------------+--------------------+----+-----------+
|   Incubation_Center|     Name_of_startup|Location of company|              Sector|city|      state|
+--------------------+--------------------+-------------------+--------------------+----+-----------+
|AIC Pinnacle Entr...|            Wastinno|  Pune, Maharashtra|         agriculture|Pune|Maharashtra|
|AIC Pinnacle Entr...|Diabetico - Rise ...|  Pune, Maharashtra|          Healthcare|Pune|Maharashtra|
|AIC Pinnacle Entr...|  3DGuru Innovations|  Pune, Maharashtra|             EduTech|Pune|Maharashtra|
|AIC Pinnacle Entr...|     Gupte Education|  Pune, Maharashtra|    Ed Tech, Defence|Pune|Maharashtra|
|AIC Pinnacle Entr...|Eldew Digital Pvt...|  Pune, Maharashtra|  IT, Virtual Events|Pune|Maharashtra|
|AIC Pinnacle Entr...|Secumatic Technol...|  Pune, Maharashtra|             Defense|Pune|Maharashtra|
|AIC Pinnacle Entr...|Catalystgreen Pri...|  Pune, Maharashtra|          E-Mobilit

In [20]:
df.filter(df.state == "Maharashtra").groupBy(df.city).count().orderBy(col('count').desc()).show()

+----------+-----+
|      city|count|
+----------+-----+
|      Pune|   14|
|    Mumbai|    9|
|   Nashik |    5|
|    Nagpur|    2|
|    Satara|    2|
|     Thane|    1|
|Ulhasnagar|    1|
+----------+-----+



In [21]:
states = ["Karnataka", "Tamilnadu", "Telangana", "Andhra Pradesh"]
df.filter(df.state.isin(states)).count()

75

In [22]:
df.filter(df.state.isin(states)).groupBy(df.state).count().orderBy(col('count')).show()

+--------------+-----+
|         state|count|
+--------------+-----+
|Andhra Pradesh|    4|
|     Telangana|   12|
|     Tamilnadu|   24|
|     Karnataka|   35|
+--------------+-----+



In [23]:
df.filter(df.state == "Gujarat").count()

7

In [24]:
df.filter((~ df.state.isin(states)) & ( df.state != "Maharashtra")).count()

65

In [25]:
 print("south state %: ",df.filter(df.state.isin(states)).count() * 100 / df.count())

south state %:  31.12033195020747


In [26]:
 print("Maharashtra %: ",df.filter(df.state == "Maharashtra").count() * 100 / df.count())

Maharashtra %:  14.107883817427386


In [27]:
 print("Gujarat %: ",df.filter(df.state == "Gujarat").count() * 100 / df.count())

Gujarat %:  2.904564315352697


In [28]:
# df.na.fill({'age': 50, 'name': 'unknown'}).show()
df.na.fill({'state': 'Unknown'}).show()

+--------------------+--------------------+--------------------+--------------------+------------+-------------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|        city|        state|
+--------------------+--------------------+--------------------+--------------------+------------+-------------+
|      ABES Ghaziabad|            Suryansh|           New Delhi|              EdTech|   New Delhi|      Unknown|
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech|      Satara|  Maharashtra|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech|      Mumbai|  Maharashtra|
|AIC Banasthali Vi...|C2M Internet Indi...|Lucknow, Uttar Pr...|         Retail Tech|     Lucknow|Uttar Pradesh|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture|        Pune|  Maharashtra|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare|        Pun